In [33]:
#IMPORTING 
from sklearn.linear_model import Ridge
import numpy as np
import sklearn.datasets
from sklearn import linear_model
import sklearn.cross_validation
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn import ensemble
import time
import random

##########
def get_tokenized_doc(document):
    from nltk.tokenize import word_tokenize
    tokenized_doc=[]
    for i in range(0,len(document)):
        word=word_tokenize(document[i])
        tokenized_doc.append(word)
    
    return tokenized_doc

##########
#Converting all words to lower case so that it is of same type at par with word to vec
def get_lower_doc(document):
    low_words=[]
    low_doc=[]
    for i in range(len(document)):
        for word in document[i]:
            lwords=word.lower()
            low_words.append(lwords)
        low_doc.append(low_words)
        low_words=[]
    return low_doc

##########
#Given a document,remove the stopwords
def get_stopwrd_free_doc(document):
    from nltk.corpus import stopwords
    stop_words = set(stopwords.words('english'))
    stpwrd_free_doc = []
    doc=[]

    for i in range (len(document)):
        for w in document[i]:
            if w not in stop_words:
                doc.append(w)
        stpwrd_free_doc.append(doc)
        doc=[]
    
    return stpwrd_free_doc

##########
#Removing non alpha words
def get_alpha_doc(document):
    alpha_doc=[]
    alpha=[]
    for i in range(len(document)):
        for word in document[i]:
            if (word.isalpha()==True):
                alpha.append(word)
        alpha_doc.append(alpha)
        alpha=[]
        
    return alpha_doc

def get_lem_doc(document):
    from nltk import WordNetLemmatizer
    from nltk.corpus import wordnet
    wnl=WordNetLemmatizer()


    lem_words=[]
    lem_doc=[]

    for i in range(len(document)):
        for word in document[i]:
            lem=wnl.lemmatize(word)
            lem_words.append(lem)
        
        lem_doc.append(lem_words)
        lem_words=[]


    return lem_doc


##########
def get_string_version(document):
    Str_data=[]
    for i in range(0,len(document)):
        val=str(document[i])
        Str_data.append(val)
    return Str_data


##########
def get_udoc(document):
    utf_doc=[]
    for i in range(0,len(document)):
        val=unicode(x[i],errors="ignore")
        utf_doc.append(val)

    return utf_doc


##########
def get_dense_matrix(term_frquency):
    from scipy.sparse import csr_matrix
    dense_trm_frq=term_frquency.todense()
    return dense_trm_frq


##########
def get_data_with_cluster_labels(filepath):
    #filepath='/Users/s0c01xa/Documents/Walmart_Bplan/Data with Cluster labels.csv'
    data_with_clust_labels=pd.read_csv(filepath)
    return data_with_clust_labels

##########
data_with_cluster_labels=get_data_with_cluster_labels('/Users/s0c01xa/Documents/Walmart_Bplan/Data with Cluster labels.csv')
#data_with_cluster_labels
#filepath='/Users/s0c01xa/Documents/Walmart_Bplan/Data with Cluster labels.csv'


##########
import pickle
filename = "/Users/s0c01xa/Documents/pkl_models hackathon/model.pkl"
#Open the filename to write 'w'
file_object = open(filename, 'r')
#dumps the model
term_frequency_model = pickle.load(file_object)


##########
# load json and create model
from keras.models import model_from_json
json_file = open('deep_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("deep_model.h5")
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

##########
deep_model=loaded_model

##########
mydata=get_data_with_cluster_labels("/Users/s0c01xa/Documents/Walmart_Bplan/Data with Cluster labels.csv")   #mydata saves the combined data with cluster labels
#x-item description is being stored and y stores the corresponding labels
#x=mydata['Combined Description']
y=mydata['Social Intent']
#Label Encoder and converting the y's into integers
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_coded=le.fit_transform(y)


##########
def get_tweet_data(tweet_filepath):
    from nltk import word_tokenize
    #filepath='/Users/s0c01xa/Documents/Walmart_Bplan/Gen_tweet_3.csv'
    f=open(tweet_filepath)
    tweet_data=f.readline()
    tweet=word_tokenize(tweet_data)
    return tweet

##########
def get_processed_tweet(tweet_filepath):
    from nltk import word_tokenize
    f=open(tweet_filepath)
    tweet_data=f.readline()
    tweet=word_tokenize(tweet_data)
    val_test=''
    for i in tweet:
        val=i
        #print val
        val_test=str(val+" "+val_test)
    return val_test

##########
def get_dense_tweet(tweet_filepath,model):
    processed_tweet=get_processed_tweet(tweet_filepath)
    test_tweet=model.transform([processed_tweet])
    dense_tweet=get_dense_matrix(test_tweet)
    return dense_tweet

##########
def get_top3_social_content(tweet_filepath,model,le,deep_model):
    #filepath=filepath
    #model=model
    tweet_dense= get_dense_tweet(tweet_filepath,model)
    similar_docs=le.inverse_transform(np.argsort(deep_model.predict_proba(tweet_dense)))
    top3_social_content=similar_docs[0][-3:]
    
    return top3_social_content
    
##########
def get_preprocessed_recco_doc(document):
    doc_basket1=get_tokenized_doc(document)
    doc_basket2=get_lower_doc(doc_basket1)
    doc_basket3=get_stopwrd_free_doc(doc_basket2)
    doc_basket4=get_alpha_doc(doc_basket3)
    doc_basket5=get_lem_doc(doc_basket4)

    return doc_basket5

##########
def get_top20_similar_items_with_tweet(social_content,tweet_filepath):
    
    tweet=get_tweet_data(tweet_filepath)
    df=get_data_with_cluster_labels('/Users/s0c01xa/Documents/Walmart_Bplan/Data with Cluster labels.csv')
    filtered_data=df[df['Social Intent']==social_content]
    doc_basket=filtered_data['Combined Description']
    doc_basket=np.array(doc_basket)

    preprocessed_recco_doc=get_preprocessed_recco_doc(doc_basket)
    #similarity basket saves the items which are similar to the tweet in most similar cluster label
    similarity_basket=[]
    for i in range(0,len(preprocessed_recco_doc)):
        common=len(set(preprocessed_recco_doc[i]).intersection( set(tweet)))
        similarity_basket.append(common)

    sim=np.argsort(similarity_basket)[-20:]
    top_twnty_sim_doc=doc_basket[sim]

    return top_twnty_sim_doc


##########
def get_top_4items_social_content(social_content,tweet_filepath):
    document=get_top20_similar_items_with_tweet(social_content,tweet_filepath)
    #document=get_top20_similarity_doc_with_tweet(cluster,tweet)
    top_4items=[]
    #Saving the particular characteristics giving the best feature size
    model=sklearn.feature_extraction.text.CountVectorizer(input='content', encoding='utf-8', decode_error='strict', 
                                                strip_accents=None, lowercase=True, preprocessor=None, tokenizer=None, 
                                                stop_words=None, ngram_range=(1,3), analyzer='word', 
                                                max_df=10000, min_df=0,max_features=1000,
                                                vocabulary=None, binary=False)

    #Transforming the data to count of token
    term_frequency=model.fit_transform(document)
    dense_term_frequency=get_dense_matrix(term_frequency)

    #Kmeans Clustering in the dense_matrix
    from sklearn.cluster import KMeans
    kclust = KMeans(n_clusters=4)
    kclust.fit(dense_term_frequency)
    clust_labels=kclust.labels_
    top_4items.append((document[clust_labels==0][0],document[clust_labels==1][0]
                       ,document[clust_labels==2][0],document[clust_labels==3][0] ))
                    
    
    return top_4items


##########
def get_top12_sim_items(tweet_filepath,model,le,deep_model):
    #filepath=tweet_filepath
    document=get_top3_social_content(tweet_filepath,model,le,deep_model)
    top12_recco=[]
    for i in document:
        #word=document[i]
        #sim_doc=get_top20_similar_items_with_tweet(document[i],tweet_filepath)
        #top4_doc=get_top_4recco(sim_doc)
        top4_doc=get_top_4items_social_content(i,tweet_filepath)
        top12_recco.append((top4_doc,('Frequency Match')))
    return top12_recco


##########
def get_gensim_item_vec(document):
    gensim_w2v=np.zeros(((len(document),50)))
    for i in range(0,len(document)):
        gensim_w2v[i]=document[i]
        
    return gensim_w2v


##########
def get_gensim_w2v_sim_matrix(social_content,tweet_filepath):
    tweet=get_processed_tweet(tweet_filepath)
    
    df=get_data_with_cluster_labels('/Users/s0c01xa/Documents/Walmart_Bplan/Data with Cluster labels.csv')
    filtered_data=df[df['Social Intent']==social_content]
    doc_basket=filtered_data['Combined Description']
    doc_basket=np.array(doc_basket)

    #document=get_top20_similar_items_with_tweet(social_content,tweet_filepath)
    from gensim.models import doc2vec
    from collections import namedtuple
    
    data=doc_basket
    docs = []
    analyzedDocument = namedtuple('AnalyzedDocument', 'words tags') 
    #converting the words to lowercase and appending it in docs in the form of tuples
    #docs[0] will contain the tuple of first doc and it's label
    for i in range (0,len(data)):
        words = data[i].lower().split()
        tags=[str(i)]
        docs.append(analyzedDocument(words, tags))

    # Train model (set min_count = 1, if you want the model to work with the provided example data set)
    model = doc2vec.Doc2Vec(docs, size = 50, window = 300, min_count = 1, workers = 5)
    gensim_item_vec=get_gensim_item_vec(model.docvecs)
    return gensim_item_vec

##########
def get_gensim_vec_tweet(tweet_filepath):
    tweet_filepath=tweet_filepath
    from gensim.models import doc2vec
    from collections import namedtuple
    tweet_doc=get_tweet_data(tweet_filepath)
    data=str(tweet_doc)
    docs = []
    analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')


    words = data.lower().split()
    tags=[str(0)]
    docs.append(analyzedDocument(words, tags))

    # Train model (set min_count = 1, if you want the model to work with the provided example data set)
    tweet_model = doc2vec.Doc2Vec(docs, size = 50, window = 300, min_count = 1, workers = 5)
    tweet_gensim_vec=tweet_model.docvecs[0]
    return tweet_gensim_vec


##########
def get_gensim_cosine_similarity(document_vec,tweet_vec):
    from scipy.spatial.distance import cosine as cs
    gensim_cosine_similarity=np.zeros(len(document_vec))
    for i in range(0,len(document_vec)):
        gensim_cosine_similarity[i]=cs(document_vec[i],tweet_vec)
    return gensim_cosine_similarity


##########
def get_gensim_item_tweet_matrix(social_content,tweet_filepath):
    #doc_social_content=doc_social_content
    #tweet_filepath=tweet_filepath
    gensim_item_matrix=get_gensim_w2v_sim_matrix(social_content,tweet_filepath)
    tweet_gensim_vec=get_gensim_vec_tweet(tweet_filepath)
    gensim_item_tweet_matrix=get_gensim_cosine_similarity(gensim_item_matrix,tweet_gensim_vec)
    return gensim_item_tweet_matrix


##########
def get_preprocessed_recco_doc(document):
    doc_basket1=get_tokenized_doc(document)
    doc_basket2=get_lower_doc(doc_basket1)
    doc_basket3=get_stopwrd_free_doc(doc_basket2)
    doc_basket4=get_alpha_doc(doc_basket3)
    doc_basket5=get_lem_doc(doc_basket4)

    return doc_basket5


##########
def get_gensim_similarity_doc_with_tweet(social_content,tweet_filepath):
    
    df=get_data_with_cluster_labels('/Users/s0c01xa/Documents/Walmart_Bplan/Data with Cluster labels.csv')
    filtered_data=df[df['Social Intent']==social_content]
    gensim_doc_basket=filtered_data['Combined Description']
    gensim_doc_basket=np.array(gensim_doc_basket)
    
    gensim_item_tweet_matrix=get_gensim_item_tweet_matrix(social_content,tweet_filepath)
    
    sim=np.argsort(gensim_item_tweet_matrix)[-4:]
    top_4_recco=gensim_doc_basket[sim]
    return top_4_recco


##########
def get_top12_gensim_sim_items(tweet_filepath,model,le,deep_model):
    document=get_top3_social_content(tweet_filepath,model,le,deep_model)
    top12_gensim_recco=[]
    for i in document:
        tweet_filepath=tweet_filepath
        gensim_sim_doc=get_gensim_similarity_doc_with_tweet(i,tweet_filepath)
        top12_gensim_recco.append((gensim_sim_doc,('Context Based')))
    return top12_gensim_recco

In [41]:
tweet_filepath="/Users/s0c01xa/Documents/Walmart_Bplan/Gen_Tweet.csv"
get_top12_gensim_sim_items(tweet_filepath,term_frequency_model,le,deep_model)

/Users/s0c01xa/Desktop/anaconda2/lib/python2.7/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


[(array([ 'SEAMLESS SLEEP SET SEAMLESS SLEEP SET NO BOUDARIES SEAMLESS SLEEP SET NO BOUNDARIES',
         "LIC TANK SET DME3 DEL WK 26 2017 LICENSE WOMEN'S AND WOMEN'S PLUS MUSCLE LICENSE",
         'ST WAFFLE KNIT ROBE DEL WK 02 2017 WAF ROBE MISMNT SECRET TREASURES',
         'LICENSE COAT SET DEL WK 44 2016 MONSTER HIGH COAT SETS MONSTER HIGH AP'], dtype=object),
  'Context Based'),
 (array(['CATSPACE WS WK 22 TEST 2016 MENS SS TEE LICENSE',
         'PEACEPNT WK39 SPN TEST FY16 MENS SS TEE LICENSE',
         'T-SHIRTS ONLINE ONLY MARILYN TEE LICENSE',
         'T-SHIRTS ONLINE ONLY MENS SS TEE LICENSE'], dtype=object),
  'Context Based'),
 (array([ 'GRAPHIC TEES#HOODIES ONLINE ONLY GIRLS HELLO KITTY ZIP VELOUR HOODIE HELLO KITTY AP',
         'SUPERMAN SOLID LIC HOODIE SUPERMAN AP',
         'MINECRAFT CRPR GLOW CREEPER GLOW BOYS TEES MINECRAFT AP',
         'POKEMON PIKACHU POKEBALL PIKACHU POKEBALL 10/12 POKEMON AP'], dtype=object),
  'Context Based')]

In [3]:
get_top20_similar_items_with_tweet('Licensed',tweet_filepath)

array(['MINNIE SS TOP DOLMAN TOP MINNIE SS PINK TOP 4/5 MINNIE MOUSE AP',
       'SNOW QUEEN TOP DISNEY FROZEN SNOW QUEEN ROCKS S/S TOP FROZEN AP',
       'SG SUB TIEFRNT TOP SPRGRL SUBL TIE FRNT SUPERGIRL SUBLIMATION TIE FRONT TOP SUPERMAN AP',
       'T-SHIRTS ONLINE ONLY MARVEL CAPTAIN AMERICA FOUR SIDED CAP BO ONLINE',
       'GRAPHIC TEES ONLINE ONLY S/S SPRGL WOW STRONG RINGER TEE SUPERMAN AP',
       'T-SHIRTS ONLINE ONLY ^^MARVEL CAPTAIN AMERICA FLAKSHIELD BOYS ONLINE',
       'NEON LOVE TEE KNIT TOP NEON LOVE S/S TEE LICENSE',
       'GRAPHIC TEES#TSHIRTS ONLINE ONLY FINDING DORY SS HI LOW CROCHET TOP FINDING DORY AP',
       'T-SHIRTS ONLINE ONLY 80S CAPTAIN RYLHTR CAPTAIN AMERICA',
       'T-SHIRTS ONLINE ONLY ^^MARVEL CAPTAIN AMERICA LOGO BOYS GRAPH ONLINE',
       'CAP AM SS HI LO TOP S3 JULY DELIVERY CPTN AMERICA SS HACCI W/LACE HANGDWN TOP CAPTAIN AMERICA AP',
       'SPIDERMAN 80S CAPTAIN BOYS POLY TEE CAPTAIN AMERICA AP',
       'SPIDERMAN 80S CAPTAIN BOYS POLY TEE CAP

In [4]:
get_top_4items_social_content('Tshirts',tweet_filepath)

[('GENERIC TANK LOVE CROCHET TANK GENERIC TOP LICENSE',
  'JR FASHION TOP HP CROSSING HARRY POTTER JR FASHION TOP HARRY POTTER',
  'LION KING HILO TANK  DISNEY TOP THE LION KING AP',
  'CAPTAIN SHIELD PANEL CAPTAIN SHIELD PANEL JUNIORS TANK TOP CAPTAIN AMERICA AP')]

In [40]:
get_top12_sim_items(tweet_filepath,term_frequency_model,le,deep_model)

[([('7502 BNP XXL ONLINE ONLY LANDAU UNISEX  SCRUB TOP ONLINE',
    'ST SLP TOP RED DEL WK 44 2016 TOP RED SECRET TREASURES',
    'CAPT AMERICA UNDEROOS TEST CAPTAIN AMERICA UNDEROOS BOXER BRIEF LAR AVENGERS AP',
    "SCRUBSTAR WOMEN'S SI ONLINE ONLY ELEC BLUE SCRUB TOP SCRUB STAR")],
  'Frequency Match'),
 ([('GENERIC TANK LOVE CROCHET TANK GENERIC TOP LICENSE',
    'CAPTAIN SHIELD PANEL CAPTAIN SHIELD PANEL JUNIORS TANK TOP CAPTAIN AMERICA AP',
    'LION KING HILO TANK  DISNEY TOP THE LION KING AP',
    'JR FASHION TOP HP CROSSING HARRY POTTER JR FASHION TOP HARRY POTTER')],
  'Frequency Match'),
 ([('CAP AM SS HI LO TOP S3 JULY DELIVERY CPTN AMERICA SS HACCI W/LACE HANGDWN TOP CAPTAIN AMERICA AP',
    'MINNIE SS TOP DOLMAN TOP MINNIE SS PINK TOP 4/5 MINNIE MOUSE AP',
    'T-SHIRTS ONLINE ONLY MARVEL CAPTAIN AMERICA FOUR SIDED CAP BO ONLINE',
    'SPIDERMAN 80S CAPTAIN BOYS POLY TEE CAPTAIN AMERICA AP')],
  'Frequency Match')]

In [35]:
get_top3_social_content(tweet_filepath,term_frequency_model,le,deep_model)

array(['Sleepwear', 'Tshirts', 'Licensed'], dtype=object)

In [34]:
#tweet=get_tweet_data(tweet_filepath)
#str_tweet=str(tweet)

In [37]:
#get_top20_similar_items_with_tweet('Licensed',tweet_filepath)